In [1]:
import pandas as pd

In [6]:
file_path_1 = r'C:\Users\16395\Downloads\final project\bigdatasource\LAPD\arrest_data\before_transformation\Arrest_Data_from_2010_to_2019.csv'
file_path_2 = r'C:\Users\16395\Downloads\final project\bigdatasource\LAPD\arrest_data\before_transformation\Arrest_Data_from_2020_to_Present.csv'

In [7]:
arrest_data_2010_to_2019 = pd.read_csv(file_path_1)
arrest_data_2020_to_present = pd.read_csv(file_path_2)


In [8]:
print("Arrest Data from 2010 to 2019:")
print(arrest_data_2010_to_2019.head())

Arrest Data from 2010 to 2019:
   Report ID Report Type Arrest Date    Time  Area ID  Area Name  \
0    2377805     BOOKING  06/22/2010  1845.0       16   Foothill   
1  121920046         RFC  09/28/2012   930.0       19    Mission   
2  101820989         RFC  10/20/2010   700.0       18  Southeast   
3   90712341         RFC  03/10/2011   940.0        7   Wilshire   
4  121909585         RFC  04/01/2012  1715.0       19    Mission   

   Reporting District  Age Sex Code Descent Code  ...  \
0                1664   46        F            W  ...   
1                1998   60        M            H  ...   
2                1829   14        M            H  ...   
3                 776   45        M            B  ...   
4                1993   37        M            H  ...   

       Disposition Description                                  Address  \
0  MISDEMEANOR COMPLAINT FILED                                PENDLETON   
1  MISDEMEANOR COMPLAINT FILED  7600    WOODMAN                    

In [9]:
print("\nArrest Data from 2020 to Present:")
print(arrest_data_2020_to_present.head())


Arrest Data from 2020 to Present:
   Report ID Report Type             Arrest Date    Time  Area ID  \
0    6636966     BOOKING  07/06/2023 12:00:00 AM  2250.0        8   
1    6637119     BOOKING  07/07/2023 12:00:00 AM  1000.0        3   
2    6624479     BOOKING  06/15/2023 12:00:00 AM  1850.0        7   
3    6636128     BOOKING  07/05/2023 12:00:00 AM  1550.0        2   
4    6636650     BOOKING  07/06/2023 12:00:00 AM  1335.0       12   

     Area Name  Reporting District  Age Sex Code Descent Code  ...  \
0      West LA                 817   46        M            B  ...   
1    Southwest                 396   39        M            B  ...   
2     Wilshire                 724   33        F            H  ...   
3      Rampart                 218   30        F            B  ...   
4  77th Street                1258   31        M            H  ...   

       Disposition Description                                  Address  \
0  MISDEMEANOR COMPLAINT FILED   900    GAYLEY        

In [11]:
combined_arrest_data = pd.concat([arrest_data_2010_to_2019, arrest_data_2020_to_present], ignore_index=False)

In [12]:
print("\nCombined Arrest Data:")
print(combined_arrest_data.head())


Combined Arrest Data:
   Report ID Report Type Arrest Date    Time  Area ID  Area Name  \
0    2377805     BOOKING  06/22/2010  1845.0       16   Foothill   
1  121920046         RFC  09/28/2012   930.0       19    Mission   
2  101820989         RFC  10/20/2010   700.0       18  Southeast   
3   90712341         RFC  03/10/2011   940.0        7   Wilshire   
4  121909585         RFC  04/01/2012  1715.0       19    Mission   

   Reporting District  Age Sex Code Descent Code  ...  \
0                1664   46        F            W  ...   
1                1998   60        M            H  ...   
2                1829   14        M            H  ...   
3                 776   45        M            B  ...   
4                1993   37        M            H  ...   

       Disposition Description                                  Address  \
0  MISDEMEANOR COMPLAINT FILED                                PENDLETON   
1  MISDEMEANOR COMPLAINT FILED  7600    WOODMAN                      AV   


In [16]:
columns_to_drop = ['Descent Code', 'Charge Group Code', 'Charge Group Description', 'Disposition Description', 'Booking Date', 'Booking Time', 'Booking Location', 'Booking Location Code']
combined_arrest_data.drop(columns=columns_to_drop, inplace=True)

In [17]:
# Data Cleaning
combined_arrest_data.fillna(method='ffill', inplace=True)
combined_arrest_data['Arrest Date'] = pd.to_datetime(combined_arrest_data['Arrest Date'], errors='coerce')
combined_arrest_data['Time'] = pd.to_numeric(combined_arrest_data['Time'], errors='coerce')
combined_arrest_data.drop_duplicates(inplace=True)

In [18]:
# Feature Engineering
combined_arrest_data['Year'] = combined_arrest_data['Arrest Date'].dt.year
combined_arrest_data['Month'] = combined_arrest_data['Arrest Date'].dt.month
combined_arrest_data['Latitude'] = combined_arrest_data['Location'].apply(lambda x: float(x.split()[1][1:]) if pd.notna(x) else None)
combined_arrest_data['Longitude'] = combined_arrest_data['Location'].apply(lambda x: float(x.split()[2][:-1]) if pd.notna(x) else None)

In [19]:
# Data Aggregation
yearly_arrests = combined_arrest_data.groupby(['Year', 'Area Name']).size().reset_index(name='Arrest Count')
age_summary = combined_arrest_data['Age'].describe()

In [20]:
# Function to clean address and cross street columns
def clean_address(address):
    if pd.isna(address):
        return address
    return ' '.join(address.split())

# Apply the cleaning function to the Address and Cross Street columns
combined_arrest_data['Address'] = combined_arrest_data['Address'].apply(clean_address)
combined_arrest_data['Cross Street'] = combined_arrest_data['Cross Street'].apply(clean_address)

In [21]:
output_file_path = r'C:\Users\16395\Downloads\final project\bigdatasource\LAPD\arrest_data\after_transformation\Combined_Arrest_Data.csv'

In [22]:
combined_arrest_data.to_csv(output_file_path, index=False)

In [23]:
print(f"Combined CSV file has been created at {output_file_path}")

Combined CSV file has been created at C:\Users\16395\Downloads\final project\bigdatasource\LAPD\arrest_data\after_transformation\Combined_Arrest_Data.csv
